In [1]:
print("Start Coding")

Start Coding


In [2]:
import cv2
import numpy as np
from PIL import Image
import os
from pathlib import Path
from tqdm import tqdm

def resize_image(image, target_size=(1024, 1024)):
    """
    Resize the image to a target size while maintaining aspect ratio.
    Args:
        image: Input image (numpy array).
        target_size: Desired size (width, height).
    Returns:
        Resized image.
    """
    h, w = image.shape[:2]
    target_w, target_h = target_size

    # Calculate the scaling factor
    scale = min(target_w / w, target_h / h)

    # Resize the image
    resized_w = int(w * scale)
    resized_h = int(h * scale)
    resized_image = cv2.resize(image, (resized_w, resized_h), interpolation=cv2.INTER_AREA)

    # Pad the image to the target size if necessary
    delta_w = target_w - resized_w
    delta_h = target_h - resized_h
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    # Pad with black pixels (or any other color)
    resized_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    return resized_image

def normalize_image(image):
    """
    Normalize the image pixel values to the range [0, 1].
    Args:
        image: Input image (numpy array).
    Returns:
        Normalized image.
    """
    return image / 255.0

def compress_image(image, quality=90):
    """
    Compress the image using JPEG compression.
    Args:
        image: Input image (numpy array).
        quality: Compression quality (0-100).
    Returns:
        Compressed image.
    """
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, compressed_image = cv2.imencode('.jpg', image, encode_param)
    compressed_image = cv2.imdecode(compressed_image, cv2.IMREAD_COLOR)
    return compressed_image

def process_high_quality_image(image_path, output_dir, target_size=(1024, 1024), compress=True, quality=90):
    """
    Process a high-quality image uniformly.
    Args:
        image_path: Path to the input image.
        target_size: Desired size for resizing.
        compress: Whether to compress the image.
        quality: Compression quality (0-100).
    Returns:
        Processed image.
    """
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Supported image formats
    supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
    
    # Get list of image files
    image_files = [
        f for f in os.listdir(image_path) 
        if os.path.splitext(f)[1].lower() in supported_formats
    ]
   
    success_count = 0
    error_count = 0
    error_files = []

    # Process each file in the input folder with progress bar
    #for filename in tqdm(image_files[0:100], desc="Resizing images", unit="image"):
    for filename in tqdm(image_files, desc="Resizing images", unit="image"):
        file_path = os.path.join(image_path, filename)
        # Load the image
        image = cv2.imread(file_path)
        #print("Image read completed")
        if image is None:
            raise ValueError("Image not found or unable to load.")
    
        # Resize the image
        resized_image = resize_image(image, target_size)
        #print("Image reszied")
    
        # Normalize the image
        normalized_image = normalize_image(resized_image)
        # Convert back to [0, 255] range for saving (this is the key step you're missing)
        image_to_save = (normalized_image * 255).astype(np.uint8)
        #print("Image normalized")
    
        # Compress the image (optional)
        if compress:
            #compressed_image = compress_image((normalized_image * 255).astype(np.uint8), quality)
            image_to_save = compress_image(image_to_save, quality)
            #return compressed_image

        #return normalized_image
        # Prepare output path
        output_path = os.path.join(output_dir, filename)
        #print(output_path)
        
        # Save resized image
        #compressed_image.save(output_path, quality=95, optimize=True)
        #cv2.imwrite(output_path, normalized_image)
        cv2.imwrite(output_path, image_to_save)
        success_count += 1
            
    return success_count, error_count, error_files
        

# Example usage
#image_path = '/kaggle/input/wrinkled-cloth-dataset/CRHD-3K_src/CRHD-3K_src/004fa773a8a4e0bf8f6f8bf242f367329e0ced97.jpg'  # Replace with your image path
#processed_image = process_high_quality_image(image_path, target_size=(512, 512), compress=True, quality=90)

if __name__ == "__main__":
    # Example paths
    input_dir_src = "/kaggle/input/wrinkled-cloth-dataset/CRHD-3K_src/CRHD-3K_src"
    output_dir_src = "./resized_images_input"
    input_dir_target = "/kaggle/input/wrinkled-cloth-dataset/CRHD-3K_gt/CRHD-3K_gt"
    output_dir_target = "./resized_images_output"

    img_size = 256
    #Example with custom size
    successes, errors, error_list = process_high_quality_image(
        input_dir_src,
        output_dir_src,
        target_size=(img_size, img_size)
    )

    successes, errors, error_list = process_high_quality_image(
        input_dir_target,
        output_dir_target,
        target_size=(img_size, img_size)
    )
    
    print(f"Successfully processed: {successes} images")
    print(f"Failed to process: {errors} images")
    if error_list:
        print("\nErrors encountered:")
        for filename, error in error_list:
            print(f"- {filename}: {error}")

# Save or display the processed image
#cv2.imwrite('processed_image.jpg', processed_image)
#cv2.imshow('Processed Image', processed_image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


Resizing images: 100%|██████████| 3022/3022 [10:58<00:00,  4.59image/s]

Successfully processed: 3022 images
Failed to process: 0 images


In [3]:
!python /kaggle/input/train-inference/data_preparation.py

Dataset split complete: 2417 training samples, 605 test samples
Training dataset size: 2417
Test dataset size: 605


In [4]:
# !python3 /kaggle/input/train-inference/model_summary.py

In [5]:
!python3 /kaggle/input/byrkbrk-model-data/train.py --dataset-name custom_data --batch-size 2 --epochs 21 --checkpoint-name custom_data_checkpoint_83.pth

/kaggle/input/byrkbrk-model-data/diffusion_model.py:343: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(os.path.join(file_dir, "checkpoints", checkpoint_nam

In [6]:
# !python3 /kaggle/input/byrkbrk-model-data/sample.py pretrained_fashion_mnist_checkpoint_49.pth --n-samples 400 --n-images-per-row 20

In [7]:
# def get_feature_target(self, target_img, feature_threshold=0.1, edge_weight=0.7, detail_weight=0.3):
#     """
#     Extract and preserve main features from the target image
    
#     Args:
#         target_img: Input target image tensor [B, C, H, W]
#         feature_threshold: Threshold for feature importance (lower = more features)
#         edge_weight: Weight given to edge features
#         detail_weight: Weight given to detail features
    
#     Returns:
#         Processed target image with main features preserved
#     """
#     batch_size = target_img.shape[0]
#     device = target_img.device
    
#     # Create output tensor
#     feature_target = torch.zeros_like(target_img)
    
#     for i in range(batch_size):
#         img = target_img[i].clone()  # [C, H, W]
        
#         # 1. Extract edges using Sobel filters
#         sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32, device=device)
#         sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32, device=device)
        
#         # Process each channel
#         edge_map = torch.zeros_like(img[0])  # [H, W]
#         for c in range(img.shape[0]):
#             # Apply sobel filters using conv2d
#             padded = F.pad(img[c].unsqueeze(0).unsqueeze(0), (1, 1, 1, 1), mode='reflect')
#             gx = F.conv2d(padded, sobel_x.unsqueeze(0).unsqueeze(0))
#             gy = F.conv2d(padded, sobel_y.unsqueeze(0).unsqueeze(0))
#             edge_strength = torch.sqrt(gx**2 + gy**2).squeeze()
#             edge_map += edge_strength
        
#         # Normalize edge map
#         if edge_map.max() > 0:
#             edge_map = edge_map / edge_map.max()
        
#         # 2. Extract high-frequency details using Laplacian filter
#         laplacian = torch.tensor([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=torch.float32, device=device)
        
#         detail_map = torch.zeros_like(img[0])  # [H, W]
#         for c in range(img.shape[0]):
#             padded = F.pad(img[c].unsqueeze(0).unsqueeze(0), (1, 1, 1, 1), mode='reflect')
#             detail = F.conv2d(padded, laplacian.unsqueeze(0).unsqueeze(0)).abs().squeeze()
#             detail_map += detail
        
#         # Normalize detail map
#         if detail_map.max() > 0:
#             detail_map = detail_map / detail_map.max()
        
#         # 3. Combine edge and detail maps
#         feature_map = edge_weight * edge_map + detail_weight * detail_map
        
#         # 4. Apply threshold to create binary feature mask
#         feature_mask = (feature_map > feature_threshold).float()
        
#         # 5. Apply mask to preserve main features
#         # Expand mask to match channels
#         feature_mask = feature_mask.unsqueeze(0).expand_as(img)
#         feature_target[i] = img * feature_mask
    
#     return feature_target